In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mat
import seaborn as sns
sns.set()
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr


In [2]:
data = pd.read_csv('/Users/sazid/Downloads/1.04.+Real-life+example.csv')

In [3]:
data.isnull().sum()

Brand             0
Price           172
Body              0
Mileage           0
EngineV         150
Engine Type       0
Registration      0
Year              0
Model             0
dtype: int64

In [4]:
data = data.dropna()

In [5]:
data.describe(include='all')


,Brand,Price,Body,Mileage,EngineV,Engine Type,Registration,Year,Model
count,4025,4025.000000,4025,4025.000000,4025.000000,4025,4025,4025.000000,4025
unique,7,NaN,6,NaN,NaN,4,2,NaN,306
top,Volkswagen,NaN,sedan,NaN,NaN,Diesel,yes,NaN,E-Class
freq,880,NaN,1534,NaN,NaN,1861,3654,NaN,188
mean,NaN,19552.308065,NaN,163.572174,2.764586,NaN,NaN,2006.379627,NaN
std,NaN,25815.734988,NaN,103.394703,4.935941,NaN,NaN,6.695595,NaN
min,NaN,600.000000,NaN,0.000000,0.600000,NaN,NaN,1969.000000,NaN
25%,NaN,6999.000000,NaN,90.000000,1.800000,NaN,NaN,2003.000000,NaN
50%,NaN,11500.000000,NaN,158.000000,2.200000,NaN,NaN,2007.000000,NaN
75%,NaN,21900.000000,NaN,230.000000,3.000000,NaN,NaN,2012.000000,NaN


In [6]:
data = data.drop(['Model'], axis=1)

In [7]:
data = data[data['Price']<=130000]
data = data[data['Mileage']<=470]
data = data[data['Year']>=1980]
data = data[data['EngineV']<7]

In [8]:
data = pd.get_dummies(data, drop_first=True)
data = data.astype(int)

In [9]:
x = data.drop(['Price'], axis=1)
y= np.log(data['Price'])

In [10]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.2, random_state=1)

In [12]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [13]:
reg.score(x_train,y_train)

0.8829407968220739

In [14]:
x = sm.add_constant(x_train)
model = sm.OLS(y_train, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     1312.
Date:                Wed, 14 May 2025   Prob (F-statistic):               0.00
Time:                        21:51:05   Log-Likelihood:                -770.02
No. Observations:                3150   AIC:                             1578.
Df Residuals:                    3131   BIC:                             1693.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.3908      0.006   1699.781      0.000       9.380       9.402
x1            -0.1156      0.008    -14.233      0.000      -0.132      -0.100
x2             0.2187      0.007     30.799      0.000       0.205       0.233
x3             0.5116      0.008     63.501      0.000       0.496       0.527
x4             0.0134      0.008      1.638      0.102      -0.003       0.029
x5             0.0009      0.009      0.108      0.914      -0.016       0.018
x6            -0.1310      0.007    -17.888      0.000      -0.145      -0.117
x7            -0.1961      0.008    -23.916      0.000      -0.212      -0.180
x8            -0.0748      0.008     -9.232      0.000      -0.091      -0.059
x9            -0.1021      0.009    -11.186      0.000      -0.120      -0.084
x10           -0.1004      0.007    -14.215      0.000      -0.114      -0.087
x11           -0.0784      0.007    -11.692      0.000      -0.092      -0.065
x12           -0.1457      0.009    -17.035      0.000      -0.163      -0.129
x13           -0.0988      0.007    -13.358      0.000      -0.113      -0.084
x14           -0.1531      0.008    -18.397      0.000      -0.169      -0.137
x15           -0.0359      0.007     -5.299      0.000      -0.049      -0.023
x16           -0.0074      0.006     -1.326      0.185      -0.018       0.004
x17           -0.0462      0.007     -6.511      0.000      -0.060      -0.032
x18            0.2543      0.006     42.673      0.000       0.243       0.266
==============================================================================
Omnibus:                      242.288   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1292.970
Skew:                          -0.106   Prob(JB):                    1.72e-281
Kurtosis:                       6.131   Cond. No.                         4.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
y_hat = reg.predict(x_train)

y_hat = pd.DataFrame(np.exp(y_hat))

train = pd.DataFrame(np.exp(y_train))

x = pd.DataFrame()
x['Predict']= y_hat.reset_index(drop=True)
x['Train']= train.reset_index(drop=True)
x['Residual']= abs(x['Train']-x['Predict'])
x['%%']= (x['Residual']*100)/x['Train']
x.describe()

,Predict,Train,Residual,%%
count,3150.000000,3150.000000,3150.000000,3150.000000
mean,16376.716574,17872.637460,4301.384540,23.942346
std,13310.081614,19146.182797,8293.738190,30.159710
min,1041.486507,600.000000,0.629420,0.025691
25%,6949.028233,6950.000000,744.792236,8.129570
50%,12379.993450,11500.000000,1860.547911,17.313851
75%,22116.244213,21000.000000,4177.473960,30.999030
max,83492.830493,129999.000000,84756.564773,629.413951


In [19]:
y_hat = reg.predict(x_test)

y_hat = pd.DataFrame(np.exp(y_hat))

train = pd.DataFrame(np.exp(y_test))

x = pd.DataFrame()
x['Predict']= y_hat.reset_index(drop=True)
x['Train']= train.reset_index(drop=True)
x['Residual']= abs(x['Train']-x['Predict'])
x['%%']= (x['Residual']*100)/x['Train']
x.describe()

,Predict,Train,Residual,%%
count,788.000000,788.000000,788.000000,788.000000
mean,16881.916744,18657.963198,4417.530599,21.867913
std,13279.649781,19732.248584,8906.938295,18.891679
min,1236.851152,1300.000000,2.335046,0.017044
25%,7692.311281,7474.250000,814.906076,8.648523
50%,13156.693123,11738.500000,1843.680005,17.226307
75%,22084.904613,21900.000000,4311.341493,29.723624
max,68858.498327,125000.000000,83264.871047,143.358292
